<center><h1>Meeting</h1></center>

---

<center>Marin Lauber</center>

### **I**nterface **Q**uasi-**N**ewton **I**nverse **L**east-**S**quare...

Standard Newton requires the Jacobian $J$, but for FSI this is unknown (or VERY exapensive)

$$
x^{k+1} = x^k + Δx^k \\
Δx^k = J\backslash-r^k
$$



Quasi-Newton Scheme say the we don't need $J$ but a good approximation $\tilde{J}^k∼ J$ is sufficient.

### How to get (learn?!) $\tilde{J}^k$?

Imagine we have some previously data $\tilde{x}^k = H(x^k)=F \circ S(xᵏ)$ and residuals $r^k=\tilde{x}^k-x^k$, we can assemble the changes between these previous data and the current data into a $W^k$ and $V^k$ matrix.

$$
V^k = [r^k-r^{k-1}, r^k-r^{k-2}, ..., r^k-r^0]\\
W^k = [\tilde{x}^k-\tilde{x}^{k-1}, \tilde{x}^k-\tilde{x}^{k-2}, ..., \tilde{x}^k-\tilde{x}^0]
$$



Then we can use $V^k$ and $W^k$ to approximate $J$, as they all satisfy the multi-secant equation
$$
\tilde{J}^{-1} V^k = W^k
$$
a Least-Square solution only is possible (overdetemined system)
$$
\tilde{J}^{-1} = W^k\left({V^k}^\top{V^k}\right)^{-1}{V^k}^\top
$$

For convergence, we aim at having $Δr = 0 - rᵏ$, which can be expressed as a combinason of the previous residuals

$$
    Δ r = 0 - r^k \approx V^kc^k
$$

We can solve this problem in a least square sense

$$
    cᵏ = \text{argmin} \left \Vert V^kc^k + rᵏ \right\Vert_2
$$
and then
$$
    x^{k+1} = x^k + W^kc^k + r^k = \tilde{x}^k + W^kc^k
$$

First step used relaxation

$$
    x^{k+1} = x^k + \omega r^k
$$

## On linear system

$$
    H(x) = x + (b - Ax)
$$
starting with $x^0=0$

<center><img src=IQN_ILS_0.svg width=1000>

starting with $x^0=b$

</center><center><img src=IQN_ILS_x0.svg width=1000></center>

## On the FSI system

Gauss-Seidel type coupling $H(x) = F∘ S(x)$, $M_\rho=$, $Ca=$

Relaxation ∼ 13, IQN ∼ 19 (iter $∼ n+1$) iterations per time steps

</center><center><img src=perpendicular_flap.gif width=800></center>



# Residuals at each time-step


</center><center><img src=iter_IQN_resid.svg width=1000></center>

In [3]:
# non-symmetric matrix wih know eigenvalues
N = 100
λ = 10 .+ (1:N)
A = triu(rand(N,N),1) + diagm(λ) # very diagonally dominant

UndefVarError: UndefVarError: `triu` not defined

### On linear system, should converge in $n+1$ iterations

<center><img src=images/GMRESvsIQNILS.png width=1000></center>

<center><h1><font color="blue">Part 2</font></h1></center>

### NURBS into WaterLily for IGA...

In [4]:
Points = [SVector(i-5, sin(i^2)) for i in 1:9]
Body = SplineBody(Points;deg=2)
sim = Simulation((8L,6L),(U,0),L;U,ν=U*L/Re,body=Body)

UndefVarError: UndefVarError: `SVector` not defined

<center><img src=images/NURBS.gif width=1000></center>

<center><img src=images/NURBS_flow_2.gif width=1000></center>

### Distance function is difficult

<center><img src=images/FastNURBS2.png width=1000></center>

Discrete distance function is better

<center><img src=images/anim_fps15.gif width=1000></center>

<center><h1>Flows with NURBS</h1></center>

<center><img src=images/circle_NURBS_flow.gif width=1000></center>

<center><img src=images/NURBS_flow_2.gif width=1000></center>

<center><img src=https://media.tenor.com/H17QEUuqMfgAAAAd/tom-scott-future-me.gif width=1000></center>></center>

<center><img src=nan_flow.gif width=1000></center>

<center><img src=nan_flow.svg width=1000></center>

In [5]:
function measure(body::SplineBody,x,t)
    # eval d=f(x,t), and n̂ = ∇f
    d = body.sdf(x,t)
    n = ForwardDiff.gradient(x->body.sdf(x,t), x)

    # correct general implicit fnc f(x₀)=0 to be a pseudo-sdf
    #   f(x) = f(x₀)+d|∇f|+O(d²) ∴  d ≈ f(x)/|∇f|
    m = √sum(abs2,n); d /= m; n /= m

    # The velocity depends on the material change of ξ=m(x,t):
    #   Dm/Dt=0 → ṁ + (dm/dx)ẋ = 0 ∴  ẋ =-(dm/dx)\ṁ
    J = ForwardDiff.jacobian(x->body.map(x,t), x)
    dot = ForwardDiff.derivative(t->body.map(x,t), t)
    return (d,n,-J\dot)
end

UndefVarError: UndefVarError: `SplineBody` not defined

$$
 \vec{f} = -\oint_{\partial\Omega} p\vec{n}\text{ d}s
$$

Hydrostatic force -> $F/\pi R^2 = 1$ (1.0683)

<center><img src=images/hydrostatic.svg width=1000></center>